# Collision Avoidance - Live Demo

Dans ce notebook, nous allons utiliser nos modèles afin de déterminer la nature de l'image qui se trouve devant le robot.  
(Suivant les classes déterminées dans les fichiers précédents)  

## Prérequis

Pour utiliser ce notebook il faut avoir au préalable réalisé les trois modèles :  
- best_model.pth  
- best_model_direction.pth  
- best_model_vitesse.pth  

Via le notebook train_model.ipynb.  
Attention à bien vérifier que le processus est terminé dans ce précédent notebook.

### Importation des bibliothèques

In [2]:
import torch
import torchvision
import time
import os
import cv2
import numpy as np
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import torch.nn.functional as F

### Définition des constantes

In [3]:
filenames = ['blocked','free','stopsign','vitesses','directions','pieton']
vitesses = ['speed1sign','speed2sign','speed3sign']
directions = ['moveright','moveleft']
filenames.sort()
vitesses.sort()
directions.sort()

largeur_image = 600
hauteur_image = 600

### Redéfinition des modèles

In [4]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, len(filenames))

model_direction = torchvision.models.alexnet(pretrained=False)
model_direction.classifier[6] = torch.nn.Linear(model_direction.classifier[6].in_features, len(directions))

model_vitesse = torchvision.models.alexnet(pretrained=False)
model_vitesse.classifier[6] = torch.nn.Linear(model_vitesse.classifier[6].in_features, len(vitesses))

### Chargement des modèles obtenus

In [5]:
model.load_state_dict(torch.load('best_model.pth'))
model_direction.load_state_dict(torch.load('best_model_direction.pth'))
model_vitesse.load_state_dict(torch.load('best_model_vitesse.pth'))

<All keys matched successfully>

La commande suivante permet de transférer le poids des modèles sur le GPU

In [6]:
device = torch.device('cuda')
model = model.to(device)
model_direction = model_direction.to(device)
model_vitesse = model_vitesse.to(device)

### Création de la fonction de preprocessing

Cette fonction permet de reformater les images obtenues par la caméra pour les faire correspondre au format des modèles. Nous réalisons alors la :

1. Conversion de BGR à RGB
2. Convertir de la disposition HWC à la disposition CHW
3. Normalisation des valeurs des différents pixels
4. Transfert des données du CPU au GPU
5. Ajout d'une diension aux valeurs

In [7]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

def preprocess2(x):
    global device, normalize
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

### Affichage de la caméra et des sliders  
Les sliders vont permettre ici de donner un aperçu du résultat de l'analyse des images qui est donnée par une certitude entre 0 et 1 pour chaque issue possible.

Par exemple, si la certitude du modèle que l'image en entrée est un piéton vaut 0.8 alors il y a de forte chance que ce soit le cas. Les sliders viennent donc afficher cette valeur et nous permet de "voir" l'interprétation du robot.


In [8]:
camera = Camera.instance(width=largeur_image, height=hauteur_image,fps=3)
image = widgets.Image(format='jpeg', width=largeur_image, height=hauteur_image)
sliders = []

for name in filenames+vitesses+directions:
    sliders.append(widgets.FloatSlider(description=name, min=0.0, max=1.0, orientation='vertical'))
    
speed_slider = widgets.FloatSlider(description='Speed', min=0.0, max=0.2, orientation='vertical')
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


box_layout = widgets.Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='80%',
                    height='',
                    flex_flow='row',
                    display='flex')
carousel = widgets.Box(children=sliders, layout=box_layout)
display(widgets.VBox([image, speed_slider]))
display(widgets.VBox([widgets.Label('Proba.:'), carousel]))


Création du robot pour le controller

In [9]:
robot = Robot()

Les fonctions suivantes permettent de gérer les différentes situations:
- D'abord, on détecte s'il y a un obstacle, un stop ou un piéton : si c'est le cas on arrête le robot
- On détecte ensuite s'il y a des panneaux vitesses ou direction
    - Si un panneau vitesse est détecté on utilise le modele_vitesse pour voir quel panneau est détecté
    - Même chose les panneaux direction avec le modele_direction
- sinon on considère que c'est libre et on avance  

Attention, vous aurez besoin de la fonction crop disponible tout en bas avant d'executer cette portion du code.

In [ ]:
# Cette première fonction permet de mettre en avant les objets détectés mais ne fait pas avancer le robot
def update(change):
    global sliders, robot
    x = change['new'] 
    x2 = change['new']
    x = preprocess(x)
    y = model(x)

    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    probs = []
    
    for i in range(len(filenames)):
        probs.append(float(y.flatten()[i]))
        sliders[i].value = probs[i]
             
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

In [20]:
#Cette seconde fonction doit permettre de mettre en évidence la distinction des panneaux vitesses
#Nous n'avons pas pu aller au bout de la démarche ici car la détection de contour n'est pas assez robuste

decisions = ["blocked", "directions","free","pieton","stopsign","vitesses"]
seuil = 0.7
speed_coef = 1
def save_snapshot():
    image_path = os.path.join('analyse/', '1.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)
         
def update(change):
    global sliders, robot, speed_coef
    x = change['new'] 
    x2 = change['new']
    x = preprocess(x)
    y = model(x)

    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    probs = []
    
    for i in range(len(filenames)):
        probs.append(float(y.flatten()[i]))
        sliders[i].value = probs[i]
        

    os.remove('analyse/1.jpg')
    save_snapshot()
    time.sleep(0.01)
    crop('analyse/1.jpg','vitesse')
    cropX = cv2.imread('analyse/1_crop.jpg')
    time.sleep(0.01)
        
    xv = preprocess2(cropX)
    v = model_vitesse(xv)
    v = F.softmax(v, dim=1)
    probsV = []
    for i in range(len(vitesses)):
        probsV.append(float(v.flatten()[i]))
        sliders[i+len(filenames)-1].value = probsV[i]            
        
    if probsV[0]>seuil:
        speed_coef = 1
    elif probsV[1]>seuil:
        speed_coef = 2
    elif probsV[2]>seuil:
        speed_coef = 3
                    
    else :
        robot.forward(speed_slider.value*speed_coef)
        
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

Pour commencer il suffit d'exécuter la ligne suivante.  
ATTENTION! Le robot peut risquer de bouger, vérifiez bien l'espace disponible autour de lui.

In [21]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

##### Arrêt
Pour arrêter le processus le code suivant sera utilisé :

In [ ]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

Pour ne plus garder la liaison vidéo (gain en bande passante), vous pouvez faire la commande suivante :

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

La ligne suivante est pour rétablir la liaison vidéo :

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

Et pour l'arrêt complet :

In [ ]:
camera.stop()

# Fonctions permettant de recadrer les images

Fonctionnement presque identique que dans train_model

In [41]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def crop(path,sign):
    
    #image_read = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    img = cv2.imread(path)
    ## conversion en couleurs hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    #filtrage
    # Set minimum and max HSV values to display
    if sign == 'vitesse':
        lower1 = np.array([0, 150, 50])
        upper1 = np.array([15,255, 140])

        lower2 = np.array([165,90, 50])
        upper2 = np.array([179, 255, 255])
        # Create HSV Image and threshold into a range.
        mask1 = cv2.inRange(hsv, lower1, upper1)
        mask2 = cv2.inRange(hsv, lower2, upper2)
        mask1 += mask2
    
    elif sign == 'direction':
        lower1 = np.array([100, 140, 50])
        upper1 = np.array([130, 255, 255])
        mask1 = cv2.inRange(hsv, lower1, upper1)
        
    target = cv2.bitwise_and(img,img, mask=mask1)



    # convert to RGB
    image = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
    imge = cv2.bitwise_not(image)
    # convert to grayscale
    gray = cv2.cvtColor(imge, cv2.COLOR_RGB2GRAY)

    # create a binary thresholded image
    if sign == 'vitesse':
        _, binary = cv2.threshold(gray, 195, 255, cv2.THRESH_BINARY_INV)
    else: 
        _, binary = cv2.threshold(gray, 250, 250, cv2.THRESH_TOZERO_INV)
           
    #recherche du contour
    contours,hierarchy=cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


    #Récupération du plus grand contour :
    indice = -1
    max = 0
    for i in range(len(contours)):
        long = len(contours[i])
        if long > max:
            indice = i
            max = long

    contour2 = [contours[indice]]

    # recadrage sur le plus grand contour:
    x = []
    y = []
    for couple in contour2[0]:
        x.append(couple[0][0])
        y.append(couple[0][1])

    xmax = -1
    xmin = 1000
    ymax = -1
    ymin = 1000
    
    #les conditions suivantes permettent de ne pas prendre en compte le contour de l'image entière qui peut apparaitre 
    for i in x:
        if 10 <i< 590 and i>xmax:
            xmax = i

        if 10<i< 590 and i<xmin:
            xmin = i
    for i in y:
        if 10 <i< 590 and i>ymax:
            ymax = i

        if 10<i< 590 and i<ymin:
            ymin = i   
    
    #Si l'image est trop petite on considère que c'est une erreur et qu'il n'y a pas de panneau sur l'image -> l'image sera alors supprimée
    if xmax-xmin < 2 or ymax-ymin<2:
        print("ok")
        return "error: No image found"
    
    #on vient rogner l'image en rajoutant 10 pixels sur chaque côté pour avoir une vue un peu plus large de l'objet
    dim = (largeur_image,hauteur_image)
    cropped_image = cv2.resize(img[ymin-10:ymax+10, xmin-10:xmax+10], dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite('analyse/1_crop.jpg', cropped_image)
    return cropped_image

ok
ok
ok
ok
ok
ok
ok
ok
